
<div style="border:1px solid black; padding:20px 20px;text-align: justify;text-justify: inter-word">
    <strong>Project - Motor Control 
    

<br/>


In [1]:
!pip install --upgrade tdmclient


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install tqdm scipy


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install numpy matplotlib pandas scipy tqdm


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


<strong>Imports :

In [4]:
import os
import sys
import math
from statistics import mean
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [190]:
import tdmclient.notebook
await tdmclient.notebook.start()

In [442]:
await tdmclient.notebook.stop()

      File "C:\Users\prona\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\iostream.py", line 267, in schedule
    self.run()
  File "C:\Users\prona\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1401, in run
    self._bootstrap_inner()
  File "C:\Users\prona\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1047, in _bootstrap_inner
    self.function(*self.args, **self.kwargs)
  File "C:\Users\prona\AppData\Local\Temp\ipykernel_45080\4043924853.py", line 16, in _run
    local_print("Exception in threading.excepthook:",
  File "C:\Users\prona\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\iostream.py", line 575, in flush
    self._event_pipe.send(b"")
    ^^^^^^^^^^^^^^^^
  File "C:\Users\prona\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\iostream.py", line 145, in _event_pipe
    self.function(*self.args, **self.kwargs)
  File "C:\Users\prona\AppData\Local\Temp\ipykernel_45080\4043924853.py"

NameError: name 'tdmclient' is not defined

Exception in thread Exception in threading.excepthook:
Exception ignored in thread started by: <bound method Thread._bootstrap of <Timer(Thread-583238, started 14876)>>
Traceback (most recent call last):
  File "C:\Users\prona\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1002, in _bootstrap
Exception in thread Thread-583239:
Traceback (most recent call last):
  File "C:\Users\prona\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1045, in _bootstrap_inner
Exception in thread Thread-583237:
Traceback (most recent call last):
  File "C:\Users\prona\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1045, in _bootstrap_inner
Exception in thread Thread-583236:
Traceback (most recent call last):
  File "C:\Users\prona\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "C:\Users\prona\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1401, in run
    self.run()
  File "C:

In [16]:
@onevent
def angle_to_rotate(pos_i, pos_f, theta_i):
    a = pos_f-pos_i
    theta_f = np.arctan2(a[1], a[0])
    r_theta = (theta_f*180)/np.pi-theta_i
    return r_theta
    
pos_thymio = np.array([1,1])
pos_goal = np.array([30,45])
theta = angle_to_rotate(pos_thymio,pos_goal,0)
print(theta)

56.61148642388849


In [17]:
@onevent
def distance_to_cover(pos_i, pos_f):
    distance = np.linalg.norm(pos_f-pos_i)
    return distance

pos_thymio = np.array([0,0])
pos_goal = np.array([30,45])
theta = angle_to_rotate(pos_thymio,pos_goal,0)
dist = distance_to_cover(pos_thymio,pos_goal)
print(dist)

54.08326913195984


In [22]:
nb_node = 3
initial_orientation = 0 #degree
positions = np.array([[0,0],[30,45],[50,50],[17,15],[50,10]]);
rotations = np.zeros(positions.shape[0]-1)
distances =  np.zeros(positions.shape[0]-1)

rotations[0] = angle_to_rotate(positions[0],positions[1],initial_orientation)
orientation = initial_orientation+rotations[0]

for i in range(1,positions.shape[0]-1):
    rotations[i] = angle_to_rotate(positions[i],positions[i+1],orientation)
    distances[i-1] = distance_to_cover(positions[i-1],positions[i])
    orientation = orientation+rotations[i]

distances[distances.size-1]=distance_to_cover(positions[distances.size-1],positions[distances.size])

print('Rotations :',rotations)
print('Distances :',distances)

Rotations : [  56.30993247  -42.27368901 -147.35155915  124.6996675 ]
Distances : [54.08326913 20.61552813 48.10405388 33.37663854]


In [219]:
print(3537/121)

29.231404958677686


## Async implementation:

In [198]:
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

Node 0102d6c0-0c6c-479f-8406-474092b0daba

In [ ]:
await node.wait_for_variables()
node.var

In [6]:
def angle_to_rotate(pos_i, pos_f, theta_i):
    a = pos_f-pos_i
    theta_f = np.arctan2(a[1], a[0])
    r_theta = (theta_f*180)/np.pi-theta_i
    return r_theta

def distance_to_cover(pos_i, pos_f):
    distance = np.linalg.norm(pos_f-pos_i)
    return distance

In [7]:
def path(positions, initial_orientation):
    nb_node = 3
    rotations = np.zeros(positions.shape[0]-1)
    distances =  np.zeros(positions.shape[0]-1)
    
    rotations[0] = angle_to_rotate(positions[0],positions[1],initial_orientation)
    orientation = initial_orientation+rotations[0]
    
    for i in range(1,positions.shape[0]-1):
        rotations[i] = angle_to_rotate(positions[i],positions[i+1],orientation)
        distances[i-1] = distance_to_cover(positions[i-1],positions[i])
        orientation = orientation+rotations[i]

    distances[distances.size-1]=distance_to_cover(positions[distances.size-1],positions[distances.size])
    rotation_ratios = rotations/360
    return distances, rotations, orientation, rotation_ratios


<strong>Kalman filter : 

In [9]:
from threading import Timer

class RepeatedTimer(object):
    def __init__(self, interval, function, *args, **kwargs):
        self._timer     = None
        self.interval   = interval
        self.function   = function
        self.args       = args
        self.kwargs     = kwargs
        self.is_running = False
        self.start()

    def _run(self):
        self.is_running = False
        self.start()
        self.function(*self.args, **self.kwargs)

    def start(self):
        if not self.is_running:
            self._timer = Timer(self.interval, self._run)
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False

In [8]:
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import numpy as np
from tqdm import tqdm

<strong> Variance for position :

In [13]:
qpx = 0.04 # variance on position state
qpy = 0.04
rpx = 0.25 # variance on position measurement 
rpy = 0.25

qpx = 0.04 # variance on position state
qpy = 0.04 
rpx = 0.25 # variance on position measurement 
rpy = 0.25


<strong>Variance for angle : 

In [14]:
std_speed = 11.77
q_theta_nu = std_speed/2 # variance on speed state
r_theta_nu = std_speed/2 # variance on speed measurement 

print(q_theta_nu,r_theta_nu)

5.885 5.885


In [264]:
# Initialising the remaining constants
# units: length [mm], time [s]
A = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
Q = np.array([[qpx, 0, 0], [0, qpy, 0],[0, 0, q_theta_nu]]); #w matrix
rotation_conv_factor = 0.41; #0.8 for 50 speed, 0.41 for 100 speed
speed_conv_factor = 0.338;

speed_conv_factor = 0.374 #mean of the two
transition_thresh = 500
l = 100 #mm, distance between the two wheels

def kalman_filter( x_est_prev, P_est_prev, Vr, Vl, Ts, camera_x=0, camera_y=0, camera_angle=0,):

    ## Prediciton through the a priori estimate
    # estimated mean of the state
    orientation = np.double(x_est_prev[2][0]%360)*2*np.pi/360
    #x_est_prev[2][0] = orientation
    B = np.array([[Ts*np.cos(orientation)/2,Ts*np.cos(orientation)/2],
         [Ts*np.sin(orientation)/2,Ts*np.sin(orientation)/2],
         [-Ts*360/(l*2*np.pi), Ts*360/(l*2*np.pi)]])
    U = np.array([[Vr*speed_conv_factor],[Vl*speed_conv_factor]])
    x_est_a_priori = np.dot(A, x_est_prev) + np.dot(B, U);
    
    # Estimated covariance of the state
    P_est_a_priori = np.dot(A, np.dot(P_est_prev, A.T));
    P_est_a_priori = P_est_a_priori + Q if type(Q) != type(None) else P_est_a_priori
    
    ## Update         
    # y, C, and R for a posteriori estimate, depending on transition
    if (camera_x != 0) : #XOR (one or the other but not both)
        # transition detected aka camera provides the position
        y = np.array([[camera_x],[camera_y],[camera_angle]])
        H = np.array([[1, 0, 0],[0, 1, 0],[0, 0, 1]])
        R = np.array([[rpx, 0, 0],[0, rpy,0],[0, 0, r_theta_nu]]) #not the value of the variance of the position measurments
        
        # innovation / measurement residual
        i = y - np.dot(H, x_est_a_priori); #measured-mean aka difference between the two states
        # measurement prediction covariance
        S = np.dot(H, np.dot(P_est_a_priori, H.T)) + R;
                 
        # Kalman gain (tells how much the predictions should be corrected based on the measurements)
        K = np.dot(P_est_a_priori, np.dot(H.T, np.linalg.inv(S)));
            
        # a posteriori estimate
        x_est = x_est_a_priori + np.dot(K,i);
        P_est = P_est_a_priori - np.dot(K,np.dot(H, P_est_a_priori));

    else:
        x_est = x_est_a_priori
        P_est = P_est_a_priori
     
    return x_est, P_est

Test of the filter

In [284]:
#test of the kalman filter
x_est = np.array([[0], [0], [0]]) # in mm !!!
#P_est = [1000 * np.ones(2)]

P_est = 0
Vr = 100;
Vl = 100;

Ts = 0.1

camera_x = 4
camera_y = 0
camera_angle = 0

#for i in range(0,20):
new_x_est, new_P_est = kalman_filter(x_est, P_est, Vr, Vl, Ts, camera_x=0, camera_y=0, camera_angle=0);
x_est = new_x_est;
P_est = new_P_est;

print(x_est[2][0])
print("new_x_est\n",new_x_est,"\n new_P_est\n", new_P_est)

8.69733149685918e-17
new_x_est
 [[3.7400000e+00]
 [0.0000000e+00]
 [8.6973315e-17]] 
 new_P_est
 [[0.04  0.    0.   ]
 [0.    0.04  0.   ]
 [0.    0.    5.885]]


In [183]:
import time

In [308]:
thymio_data = []
initial_orientation = 0 #degree
positions = np.array([[0,0],[200,150]])#[0,0],[30,45],[50,50],[17,15],[50,10]
angle_threshold = 5 #degrees
x_est = np.array([[positions[0][0]], [positions[0][1]], [initial_orientation]])
P_est = 0
Ts = 0.1
count = 0
read = 1


async def main():    

    global x_est, P_est, Ts, read
    TURN = 1
    STRAIGHT = 0
    STATE = 0
    await node.wait_for_variables() # wait for Thymio variables values
    rt = RepeatedTimer(Ts, filter)#, x_est, P_est, Ts)
    
    try:
        for i in range(0,20):
            angle = angle_to_rotate([x_est[0][0],x_est[1][0]],positions[1],x_est[2][0]);
            if (abs(angle)<angle_threshold):
                drive_task = asyncio.create_task(drive_for_seconds(client, Ts))
                await drive_task
    
            else:
                if angle>0:
                    right = 1
                else:
                    right = 0

                turn_task = asyncio.create_task(turn_for_seconds(client, Ts, right))
                await turn_task
    finally:
        rt.stop()
        node.send_set_variables(motors(0, 0))

await main()

to turn 36.86989764584402
orientation :  0
-26 24
-26 24
-26 24
-26 24
-26 24
-26 24
-26 24
-26 24
-26 24
I wait for turn
to turn 27.18556887680113
orientation :  9.642879692051753
-26 24
0 0
0 0
0 0
0 0
0 0
-93 94
-93 94
to turn 18.099417029265823
orientation :  18.728615224118293
-96 95
-96 95
-96 95
to turn 5.817714501141005
orientation :  31.007215365330858
-41 17
-43 57
-43 57
to turn 0.28603906127240464
orientation :  36.5357997221072
aaaaaaaa 0.28603906127240464
1
-76 72
-76 72
-76 72
-76 72
-76 72
-76 72
-76 72
-76 72
-76 72
-76 72
I wait for straight
to turn -31.387571701556332
orientation :  68.25015959818855
-76 72
-76 72
-76 72
-76 72
-76 72
-76 72
-76 72
-76 72
-76 72
I wait for turn
to turn -59.82488558752676
orientation :  96.79308348666176
-76 72
0 0
0 0
to turn -62.98158971372522
orientation :  99.96451947426989
79 -73
79 -73
79 -73
to turn -53.27633726211503
orientation :  90.19306805299077
64 -61
64 -61
64 -61
to turn -45.27024259868598
orientation :  82.157334976280

Main to test things :

In [326]:
thymio_data = []
initial_orientation = 0 #degree
positions = np.array([[0,0],[200,150]])#[0,0],[30,45],[50,50],[17,15],[50,10]
angle_threshold = 5 #degrees
x_est = np.array([[positions[0][0]], [positions[0][1]], [initial_orientation]])
P_est = 0
Ts = 0.1
count = 0
read = 1


async def main():    

    global x_est, P_est, Ts, read
    TURN = 1
    STRAIGHT = 0
    STATE = 0
    await node.wait_for_variables() # wait for Thymio variables values
    rt = RepeatedTimer(Ts, filter)#, x_est, P_est, Ts)
    
    try:
        for i in range(0,20):
            angle = angle_to_rotate([x_est[0][0],x_est[1][0]],positions[1],x_est[2][0]);
            print("to turn", angle)
            print("orientation : ",x_est[2][0])
            if (abs(angle)<angle_threshold) and (STATE == TURN):
                print("aaaaaaaa", abs(angle))
                print(STATE)
                read = 0
                time.sleep(10)
                read = 1
                STATE = STRAIGHT
                print("I wait for straight")
            elif (abs(angle)<angle_threshold) and (STATE == STRAIGHT):
                #straight for Ts=0.1 seconds
                print("straight")
                drive_task = asyncio.create_task(drive_for_seconds(client, Ts))
                await drive_task
    
            else:
                if STATE == STRAIGHT:
                    read = 0
                    time.sleep(1)
                    read = 1
                    print("I wait for turn")
                    STATE = TURN
                else:
                    if angle>0:
                        right = 1
                    else:
                        right = 0
    
                    turn_task = asyncio.create_task(turn_for_seconds(client, Ts, right))
                    await turn_task
    finally:
        rt.stop()
        node.send_set_variables(motors(0, 0))

await main()

to turn 36.86989764584402
orientation :  0
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
I wait for turn
to turn 3.727084395949582
orientation :  33.17150614065803
aaaaaaaa 3.727084395949582
1
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
I wait for straight
to turn -339.05493307364975
orientation :  375.9437362607903
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-85 87
-8

In [21]:
def motors(left, right):
    return {
        "motor.left.target": [left],
        "motor.right.target": [right],
    }

In [323]:
import asyncio
import time
    
async def drive_for_seconds(client, seconds):
    # Set the motor speeds
    v = {
        "motor.left.target": [int(99)],
        "motor.right.target": [int(100)],
    }
    await node.set_variables(v)
    await asyncio.sleep(seconds)

    #to delete probably 
    vstop = {
        "motor.left.target": [int(0)],
        "motor.right.target": [int(0)],
    }
    await node.set_variables(vstop)


async def turn_for_seconds(client, seconds,right):
    # Set the motor speeds
    if seconds != 0:
        if right==1:
            v = {
                "motor.left.target": [int(100)],
                "motor.right.target": [int(-100)],
            }
        else:
            v = {
                "motor.left.target": [int(-100)],
                "motor.right.target": [int(100)],
            }
        await node.set_variables(v)
    await asyncio.sleep(abs(seconds))

    #to delete probably
    vstop = {
        "motor.left.target": [int(0)],
        "motor.right.target": [int(0)],
    }
    await node.set_variables(vstop)

In [316]:
def filter():#x_est, P_est, Ts):
    global x_est, P_est, Ts, read
    #thymio_data = []
    #print(x_est)
    #if read = 1:
    thymio_data.append({"left_speed":node["motor.left.speed"],
                        "right_speed":node["motor.right.speed"]})
    Vl = thymio_data[-1]["left_speed"]
    Vr = thymio_data[-1]["right_speed"]
    #else: 
    #    Vr = 0
    #    Vl = 0

    print(Vr, Vl)
    
    x_est, P_est = kalman_filter(x_est, P_est, Vr, Vl, Ts, camera_x=0, camera_y=0, camera_angle=0);
    #print(x_est)
    

In [ ]:
node.send_set_variables(motors(0, 0))
